In [7]:
def quantify_results(Method, direc, SaveName, frameresult, OrigFrameFolder, ending):
        from tqdm.notebook import tqdm, trange
        from matplotlib.pyplot import imshow, subplots
        import cv2
        import numpy as np
        import glob
        import os
        import pandas as pd
        from skimage.measure import compare_ssim as ssim
        from skimage.measure import compare_mse as mse
        from skimage.measure import compare_psnr as psnr
        from PIL import Image

        #for cross change all ellipse to cross
        columnNames=["videoName", "Method", "PSNRavg","PSNRstd", "SSIMavg","SSIMstd", "MSEavg","MSEstd"]
        df = pd.DataFrame(columns=columnNames)            

        videoNames=[f'hyperK_{i}' for i in range(343, 373)]



        for videoName in tqdm(videoNames):
                # print(direc+videoName+frameresult+"/*"+ending)
                TransFrameFiles=sorted(glob.glob(direc+videoName+frameresult+"/*"+ending))
                OrigFrameFile=OrigFrameFolder+videoName+"/"
                PSNR=[]
                MSE=[]
                SSIM=[]
                cnt=0
                adder=0
                for TransFrameFile in TransFrameFiles:
                        cnt=cnt+1
                        numStr=int(''.join(filter(str.isdigit, os.path.basename(TransFrameFile))))
                        if cnt==1 and numStr==1: adder=1
                        else: adder=0
                        TransName=(str(numStr-adder)).zfill(5)+".jpg"
                        TransImageFull= np.array(Image.open(TransFrameFile))

                        OrigImage=np.array(Image.open(OrigFrameFile+TransName))
                        # print(OrigImage.shape, TransImageFull.shape)

                        PSNR.append(psnr(OrigImage,TransImageFull))
                        MSE.append(mse(OrigImage,TransImageFull))
                        SSIM.append(ssim(OrigImage,TransImageFull,multichannel=3))


                PSNRavg=np.mean(PSNR)
                PSNRstd=np.std(PSNR)

                MSEavg=np.mean(MSE)
                MSEstd=np.std(MSE)

                SSIMavg=np.mean(SSIM)
                SSIMstd=np.std(SSIM)

                df_temp=pd.DataFrame([[videoName, Method,
                                PSNRavg, PSNRstd, SSIMavg,SSIMstd, MSEavg,MSEstd]],columns=columnNames)
                df=df.append(df_temp)

        df.to_csv(SaveName)  

# Examples

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
OrigFrameFolder = "/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/"
savedir = "/home/rdaher/workspace/phd-work/Endo-STTN/results/"

direcbase=["/home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/",
           "/home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/"]
Method = ["sttn_SanityCheck16",
          "sttn_hyper-kvasir-Addingspec-GTdata"]#,"PseudoSTTNdataCycle_Identity_A2losses_"]
iters=["gen_00020",
       "gen_00003"]
frameresult=["/notoverlaid/nomask/B/frameresult/",
             "/notoverlaid/notshifted/nomask/frameresult/"]
ending = ["",""]#,"_fake_B.png"]


for i in range(len(Method)):
    direc=direcbase[i]+Method[i]+"/NS/"+iters[i]+"/"
    SaveName=savedir+"ResultsInpainting"+Method[i]+".csv"
    quantify_results(Method[i], direc, SaveName, frameresult[i], OrigFrameFolder)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


IndexError: list index out of range

In [8]:
# nontemporal
%matplotlib inline
%load_ext autoreload
%autoreload 2
OrigFrameFolder = "/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/"
savedir = "/home/rdaher/workspace/phd-work/Endo-STTN/results/"

direcbase=["/home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/",
           "/home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/"]
Method = ["sttn_SanityCheck16",
          "sttn_hyper-kvasir-Addingspec-GTdata"]#,"PseudoSTTNdataCycle_Identity_A2losses_"]
MethodT = ["sttn_SanityCheck16-nontemp",
          "sttn_hyper-kvasir-Addingspec-GTdata-nontemp"]#,"PseudoSTTNdataCycle_Identity_A2losses_"]
iters=["gen_00020",
       "gen_00003"]
frameresult=["/notoverlaid/nomask/B/frameresult/",
             "/notoverlaid/notshifted/nomask/frameresult/"]
ending = ["",""]#,"_fake_B.png"]


for i in range(len(Method)):
    direc=direcbase[i]+Method[i]+"/NS/NoTemporalv3/"+iters[i]+"/"
    SaveName=savedir+"ResultsInpaintingNoTemporal"+Method[i]+".csv"
    quantify_results(MethodT[i], direc, SaveName, frameresult[i], OrigFrameFolder, ending[i])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#cycleGAN
%matplotlib inline
%load_ext autoreload
%autoreload 2
OrigFrameFolder = "/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/"
savedir = "/home/rdaher/workspace/phd-work/Endo-STTN/results/"

direcbase=["/home/rdaher/workspace/phd-work/Recycle-GAN/results/",
           "/home/rdaher/workspace/phd-work/Recycle-GAN/results/"]
Method = ["PseudoSTTNdataCycleGAN",
          "PseudoSTTNdataCycleGAN_identityAhole"]
iters=["/test_1/images",
       "/test_1/images"]
frameresult=["",
             ""]
ending = ["_fake_B.png",
          "_fake_B.png"]


for i in range(len(Method)):
    direc=direcbase[i]+Method[i]+iters[i]+"/"
    SaveName=savedir+"ResultsInpainting"+Method[i]+".csv"
    quantify_results(Method[i], direc, SaveName, frameresult[i], OrigFrameFolder, ending[i])


# Before Evaluation

1. To generate results before evaluating:
    1. make sure in test code:
        - added for memory issue  
        if len(frames)>927:  
            masks=masks[:927]  
            frames=frames[:927]

In [1]:
#CycleSTTN
import glob
import os
os.chdir("/home/rdaher/workspace/phd-work/Endo-STTN/")
for videoDir in glob.glob("/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/*"):
    videoName = os.path.basename(videoDir)
    cmd = ["python test-cycle.py --gpu 0 " +
        "--NoMask --output /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/NS/ " +
        "--frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/"+videoName+"/ " +
        "--mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/"+videoName+"/ " +
        "--ckptpath /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/ " +
        "--ckptnumber 20"][0]
    print(cmd)
    os.system(cmd)

python test-cycle.py --gpu 0 --NoMask --output /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/NS/ --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/hyperK_351/ --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/hyperK_351/ --ckptpath /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/ --ckptnumber 20
python test-cycle.py --gpu 0 --NoMask --output /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/NS/ --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/hyperK_359/ --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/hyperK_359/ --ckptpath /home/datasets/Trainedmodelsandresults/Endo-STTN

In [1]:
#no temporal CycleSTTN
import glob
import os
os.chdir("/home/rdaher/workspace/phd-work/Endo-STTN/")
cmd = ["python test-cycle-notemp-v3.py --gpu 4 " +
    "--NoMask --output /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/NS/NoTemporalv3/ " +
    "--frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/ " +
    "--mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/ " +
    "--ckptpath /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/ " +
    "--ckptnumber 20"][0]
print(cmd)
os.system(cmd)

python test-cycle-notemp-v3.py --gpu 4 --NoMask --output /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/NS/NoTemporalv3/ --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/ --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/ --ckptpath /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/ --ckptnumber 20


In [1]:
## VERY SLOWWW
# #no temporal CycleSTTN
# import glob
# import os
# os.chdir("/home/rdaher/workspace/phd-work/Endo-STTN/")
# list_out = []
# videoNames=[f'hyperK_{i}' for i in range(343, 373)]
# Method  = "/home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/"
# output = Method + "NS/notemporal/"

# for videoName in videoNames:
#     FramesPath=glob.glob("/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/"+videoName+"/*.jpg")
#     for FramePath in FramesPath:
#         FrameName=os.path.basename(FramePath)
#         filename, file_extension = os.path.splitext(FrameName)

#         if int(filename)<927:
#             out_str = ["python test-cycle-notemp.py --gpu 0 " +
#                 " --NoMask --output " + output +"/" +
#                 " --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/"+videoName +"/"+FrameName+
#                 " --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/"+videoName +"/"+FrameName+
#                 " --ckptpath " + Method +
#                 " --ckptnumber 20"][0]
#             list_out = list_out + [out_str]

# # for i in range(len(FramesPath)):
# #     print(list_out[i])
# #     os.system(list_out[i])

# import pandas as pd
# pd.DataFrame(list_out).to_csv(r'runResults-NoTemporalCycleSTTN.sh', header=None, index=None, sep=',', mode='a') 
# #then run bash script

In [1]:
#STTN
import glob
import os
os.chdir("/home/rdaher/workspace/phd-work/Endo-STTN/")
for videoDir in glob.glob("/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/*"):
    videoName = os.path.basename(videoDir)
    cmd = ["python test.py --gpu 0 " +
        " --output /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/NS/ " +
        " --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/"+videoName+"/ " +
        " --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/"+videoName+"/ " +
        " -c /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/ " +
        " -cn 3 " + 
        " --nomask"][0]
    print(cmd)
    os.system(cmd)

python test.py --gpu 0  --output /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/NS/  --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/hyperK_351/  --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/hyperK_351/  -c /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/  -cn 3  --nomask
python test.py --gpu 0  --output /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/NS/  --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/hyperK_359/  --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/hyperK_359/  -c /home/datasets/Trainedmodelsand

In [2]:
#no temp STTN
import glob
import os
os.chdir("/home/rdaher/workspace/phd-work/Endo-STTN/")
cmd = ["python test-notemp-v3.py --gpu 5 " +
    " --output /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/NS/NoTemporalv3/ " +
    " --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/ " +
    " --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/ " +
    " -c /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/ " +
    " -cn 3 " + 
    " --nomask"][0]
print(cmd)
os.system(cmd)

python test-notemp-v3.py --gpu 5  --output /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/NS/NoTemporalv3/  --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/  --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/  -c /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/  -cn 3  --nomask


0

In [2]:
## TOO SLOW
# #no temporal STTN
# import glob
# import os
# os.chdir("/home/rdaher/workspace/phd-work/Endo-STTN/")
# list_out = []
# videoNames=[f'hyperK_{i}' for i in range(343, 373)]
# Method  = "/home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/"
# output = Method + "NS/notemporal/"

# for videoName in videoNames:
#     FramesPath=glob.glob("/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/"+videoName+"/*.jpg")
#     for FramePath in FramesPath:
#         FrameName=os.path.basename(FramePath)
#         filename, file_extension = os.path.splitext(FrameName)
#         if int(filename)<927:
#             out_str = ["python test-notemp.py --gpu 0 " +
#                 " --output " + output +"/" +
#                 " --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFramesNS/"+videoName +"/"+FrameName+
#                 " --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/"+videoName +"/"+FrameName+
#                 " -c " + Method +
#                 " -cn 3 " + 
#                 " --nomask"][0]
#             list_out = list_out + [out_str]

# # for i in range(len(FramesPath)):
# #     print(list_out[i])
# #     os.system(list_out[i])

# import pandas as pd
# pd.DataFrame(list_out).to_csv(r'runResults-NoTemporalSTTN.sh', header=None, index=None, sep=',', mode='a') 
# #then run bash script

In [ ]:
#CycleGAN
# python test.py --dataroot /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/ \
#  --name PseudoSTTNdataCycle_IdentityAonly_A2losses_ \
#  --model reCycle_gan --which_model_netG resnet_6blocks  --which_model_netP unet_256 --dataset_mode aligned_triplet --no_dropout \
#  --resize_or_crop scale_width --fineSize 256 --loadSize 256 --display_id 5 --which_epoch 2 --gpu 2


In [1]:
#STTN
import glob
import os
os.chdir("/home/rdaher/workspace/phd-work/Endo-STTN/")
for videoDir in glob.glob("/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/*"):
    videoName = os.path.basename(videoDir)
    cmd = ["python test.py --gpu 0 " +
        " --output /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/ " +
        " --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/"+videoName+"/ " +
        " --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/"+videoName+"/ " +
        " -c /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/ " +
        " -cn 3 " + 
        " --nomask"][0]
    print(cmd)
    os.system(cmd)

python test.py --gpu 0  --output /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/  --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/hyperK_351/  --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/hyperK_351/  -c /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/  -cn 3  --nomask
python test.py --gpu 0  --output /home/datasets/Trainedmodelsandresults/Endo-STTN/release_model/sttn_hyper-kvasir-Addingspec-GTdata/  --frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/hyperK_359/  --mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/hyperK_359/  -c /home/datasets/Trainedmodelsandresults/En